In [ ]:
# With ipython notebooks
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=""

# With python
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ['CUDA_VISIBLE_DEVICES'] = "0,3"  # specify which GPU(s) to be used

# With bash
# export CUDA_DEVICE_ORDER="PCI_BUS_ID"
# export CUDA_VISIBLE_DEVICES="0,3" 

import os
import numpy as np
import tensorflow as tf
# tf.enable_eager_execution()
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior() 
# tf.enable_eager_execution()


from albert_act import (
    modeling, 
    optimization, 
    tokenization
)

# IPython automatically reload all changed code
%load_ext autoreload
%autoreload 2

In [ ]:
tf.__version__

In [ ]:
# tf_upgrade_v2   --intree ./albert   --outtree ./albert_v2/   --reportfile report.txt

In [ ]:
albert_config = modeling.AlbertConfig.from_json_file('./albert_config.json')

In [ ]:
input_ids = tf.constant([[    2,    17, 11900,    14,     8,   772,    11,     8, 20797, 14527, 12,    95,  1096,  2365,    76,     3]], dtype=tf.int32)
input_mask = tf.constant([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=tf.int32)
segment_ids = tf.constant([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=tf.int32)
# input_ids = tf.compat.v1.placeholder(tf.int32, [None, None], "input_ids")
# input_mask = tf.compat.v1.placeholder(tf.int32, [None, None], "input_mask")
# segment_ids = tf.compat.v1.placeholder(tf.int32, [None, None], "segment_ids")

model = modeling.AlbertModelWithACT(
    config=albert_config,
    is_training=False,
    input_ids=input_ids,
    input_mask=input_mask,
    token_type_ids=segment_ids,
    use_one_hot_embeddings=False)


In [ ]:
model.get_n_updates()

In [ ]:
tvars = tf.trainable_variables()

In [ ]:
reader = tf.train.load_checkpoint("./model.ckpt-112500")
shapes = reader.get_variable_to_shape_map()
dtypes = reader.get_variable_to_dtype_map()

In [ ]:
# tf.train.list_variables("./model.ckpt-112500")
reader

In [ ]:
tvars

In [ ]:
INIT_CHECKPOINT = "./model.ckpt-112500"

In [ ]:
(assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(
              tvars, INIT_CHECKPOINT, 1)

In [ ]:
tf.train.init_from_checkpoint(INIT_CHECKPOINT, assignment_map[0])

In [ ]:
model.get_n_updates()

In [ ]:
tf.global_variables()

In [ ]:
with tf.Session() as sess:
  # input_ids = tf.Variable(tf.constant([[    2,    17, 11900,    14,     8,   772,    11,     8, 20797, 14527, 12,    95,  1096,  2365,    76,     3]], dtype=tf.int32))
  # input_mask = tf.Variable(tf.constant([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=tf.int32))
  # segment_ids = tf.Variable(tf.constant([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=tf.int32))
  # init = tf.global_variables_initializer()
  # tf.train.init_from_checkpoint(INIT_CHECKPOINT, assignment_map[0])
  # tf.global_variables_initializer().run()
  tf.initialize_variables(var_list=tf.trainable_variables()).run()
  
  # sess.run(init)
  # print(tvars[0].eval())
  
  # print(sess.run('bert/embeddings/word_embeddings:0'))
  # print(sess.run('bert/encoder/transformer/ffn_1/intermediate/dense/kernel:0'))
  # print(sess.run('bert/encoder/transformer/sigmoid_activation_for_pondering/dense/kernel:0'))
  print(model.get_n_updates().eval())
  # print(model.embedding_output.eval())
  # print(model.sequence_output.eval())
  


In [ ]:
# with tf.Session() as sess:
#   new_saver = tf.train.import_meta_graph('./model.ckpt-112500.meta')
#   new_saver.restore(sess, './model.ckpt-112500')
# not working since tpu checkpoint. Should be converted

In [ ]:
tf.compat.v1.disable_eager_execution()

input_ids = tf.compat.v1.placeholder(tf.int32, [None, None], "input_ids")
input_mask = tf.compat.v1.placeholder(tf.int32, [None, None], "input_mask")
segment_ids = tf.compat.v1.placeholder(tf.int32, [None, None], "segment_ids")
mlm_positions = tf.compat.v1.placeholder(tf.int32, [None, None], "mlm_positions")

model = modeling.AlbertModelWithACT(
    config=albert_config,
    is_training=False,
    input_ids=input_ids,
    input_mask=input_mask,
    token_type_ids=segment_ids,
    use_one_hot_embeddings=False)

In [ ]:
tvars = tf.compat.v1.trainable_variables()

In [ ]:
INIT_CHECKPOINT = "./model.ckpt-112500"

In [ ]:
(assignment_map, initialized_variable_names
) = modeling.get_assignment_map_from_checkpoint(tvars, INIT_CHECKPOINT)

In [ ]:
# ckpt_reader = tf.train.NewCheckpointReader(INIT_CHECKPOINT)

In [ ]:
tf.compat.v1.train.init_from_checkpoint(INIT_CHECKPOINT, assignment_map)

In [ ]:
init = tf.compat.v1.global_variables_initializer()

In [ ]:
sess = tf.compat.v1.InteractiveSession()

In [ ]:
sess.run(init)

In [ ]:
tvars[18].name.strip()[:-2]

In [ ]:
tvars[18].eval()

In [ ]:
for tv in tvars:
  file_name = os.path.join('./weights/', '{}.npy'.format(tv.name.strip()[:-2].replace('/', '-')))
  print(file_name)
  with open(file_name, 'wb') as f:
    np.save(f, tv.eval())

In [ ]:
# with tf.Session() as sess:
#   sess.run(init)
#   print(tvars[0].eval())

In [ ]:
# with tf.Session() as sess:  print(tvars[0].eval()) 

In [ ]:
initialized_variable_names